# Introduction to Notebook 
This notebook shows a handson session of the application of Topic Modelling using Gensim, a popular python library. The notebook will go through how to import an API based dataset, how to apply Latent Semantic Analysis (LSA) and Latent Dirichlet Allocation (LDA)​ algorithm and how to measure the accuracy of the topic modelling model using the coherence score. 



# Dataset
The dataset that will be used as a sample in this notebook is the [Twenty Newsgroups Data](https://archive.ics.uci.edu/ml/datasets/Twenty+Newsgroups) Set from the open source UCI Machine Learning Repository. For the purpose of this exercise, a cleaned version of this dataset will be imported from the [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html?highlight=newsgroup#sklearn.datasets.fetch_20newsgroups) API

In [1]:
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
import pandas as pd

In [2]:
dataset = fetch_20newsgroups(subset = 'all',shuffle= False, random_state=32,remove=('headers', 'footers', 'qutes'))

In [3]:
dataset.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [4]:
dataset['data']

["gajarsky@pilot.njin.net writes:\n\nmorgan and guzman will have era's 1 run higher than last year, and\n the cubs will be idiots and not pitch harkey as much as hibbard.\n castillo won't be good (i think he's a stud pitcher)",
 'Well, I just got my Centris 610 yesterday.  It took just over two \nweeks from placing the order.  The dealer (Rutgers computer store) \nappologized because Apple made a substitution on my order.  I ordered\nthe one without ethernet, but they substituted one _with_ ethernet.\nHe wanted to know if that would be "alright with me"!!!  They must\nbe backlogged on Centri w/out ethernet so they\'re just shipping them\nwith!  \n\n\tAnyway, I\'m very happy with the 610 with a few exceptions.  \nBeing nosy, I decided to open it up _before_ powering it on for the first\ntime.  The SCSI cable to the hard drive was only partially connected\n(must have come loose in shipping).  No big deal, but I would have been\npissed if I tried to boot it and it wouldn\'t come up!\n\tTh

In [5]:
pprint(dataset.target_names)

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


In [6]:
len(dataset.target_names)

20

In [7]:
dataset.target.shape

(18846,)

In [8]:
dataset.filenames.shape

(18846,)

In [9]:
dataset_df = pd.DataFrame({'News':dataset.data, 'Label' : dataset.target})

In [10]:
dataset_df

,News,Label
0,gajarsky@pilot.njin.net writes:\n\nmorgan and ...,9
1,"Well, I just got my Centris 610 yesterday. It...",4
2,Archive-name: cryptography-faq/part10\nLast-mo...,11
3,> ATTENTION: Mac Quadra owners: Many storage i...,4
4,bobbe@vice.ICO.TEK.COM (Robert Beauchaine) wri...,0
...,...,...
18841,\nWhy are circuit boards green? The material ...,12
18842,In article <1r941o$3tu@menudo.uh.edu> inde7wv@...,8
18843,We were told that the resolution on the 5FGe c...,4
18844,CAD Setup For Sale:\n\nG486PLB Local Bus Mothe...,6


In [11]:
dataset_df['Label_name'] = dataset_df['Label'].apply(lambda x: dataset.target_names[x])

In [12]:
dataset_df

,News,Label,Label_name
0,gajarsky@pilot.njin.net writes:\n\nmorgan and ...,9,rec.sport.baseball
1,"Well, I just got my Centris 610 yesterday. It...",4,comp.sys.mac.hardware
2,Archive-name: cryptography-faq/part10\nLast-mo...,11,sci.crypt
3,> ATTENTION: Mac Quadra owners: Many storage i...,4,comp.sys.mac.hardware
4,bobbe@vice.ICO.TEK.COM (Robert Beauchaine) wri...,0,alt.atheism
...,...,...,...
18841,\nWhy are circuit boards green? The material ...,12,sci.electronics
18842,In article <1r941o$3tu@menudo.uh.edu> inde7wv@...,8,rec.motorcycles
18843,We were told that the resolution on the 5FGe c...,4,comp.sys.mac.hardware
18844,CAD Setup For Sale:\n\nG486PLB Local Bus Mothe...,6,misc.forsale


# Dataset preprocessing

In [13]:
%%capture
!pip install -U gensim

In [14]:
from gensim.utils import tokenize
from gensim.parsing.preprocessing import preprocess_string,strip_tags,strip_punctuation,strip_numeric,remove_stopwords,strip_short
from gensim.corpora.dictionary import Dictionary
from gensim import models

In [15]:
help(preprocess_string)

Help on function preprocess_string in module gensim.parsing.preprocessing:

preprocess_string(s, filters=[<function <lambda> at 0x7f1327fb8f80>, <function strip_tags at 0x7f1328196f80>, <function strip_punctuation at 0x7f1328196b90>, <function strip_multiple_whitespaces at 0x7f1356e0ee60>, <function strip_numeric at 0x7f1356e0e830>, <function remove_stopwords at 0x7f1356e16680>, <function strip_short at 0x7f1356e0e170>, <function stem_text at 0x7f1327fb8dd0>])
    Apply list of chosen filters to `s`.
    
    Default list of filters:
    
    * :func:`~gensim.parsing.preprocessing.strip_tags`,
    * :func:`~gensim.parsing.preprocessing.strip_punctuation`,
    * :func:`~gensim.parsing.preprocessing.strip_multiple_whitespaces`,
    * :func:`~gensim.parsing.preprocessing.strip_numeric`,
    * :func:`~gensim.parsing.preprocessing.remove_stopwords`,
    * :func:`~gensim.parsing.preprocessing.strip_short`,
    * :func:`~gensim.parsing.preprocessing.stem_text`.
    
    Parameters
    -------

In [16]:
dataset_df['Clean_news'] = dataset_df['News'].apply(preprocess_string)

In [17]:
dataset_df

,News,Label,Label_name,Clean_news
0,gajarsky@pilot.njin.net writes:\n\nmorgan and ...,9,rec.sport.baseball,"[gajarski, pilot, njin, net, write, morgan, gu..."
1,"Well, I just got my Centris 610 yesterday. It...",4,comp.sys.mac.hardware,"[got, centri, yesterdai, took, week, place, or..."
2,Archive-name: cryptography-faq/part10\nLast-mo...,11,sci.crypt,"[archiv, cryptographi, faq, modifi, faq, sci, ..."
3,> ATTENTION: Mac Quadra owners: Many storage i...,4,comp.sys.mac.hardware,"[attent, mac, quadra, owner, storag, industri,..."
4,bobbe@vice.ICO.TEK.COM (Robert Beauchaine) wri...,0,alt.atheism,"[bobb, vice, ico, tek, com, robert, beauchain,..."
...,...,...,...,...
18841,\nWhy are circuit boards green? The material ...,12,sci.electronics,"[circuit, board, green, materi, goe, name, cir..."
18842,In article <1r941o$3tu@menudo.uh.edu> inde7wv@...,8,rec.motorcycles,"[articl, indewv, rosi, edu, write, bike, lucki..."
18843,We were told that the resolution on the 5FGe c...,4,comp.sys.mac.hardware,"[told, resolut, fge, anybodi, tri, run, higher..."
18844,CAD Setup For Sale:\n\nG486PLB Local Bus Mothe...,6,misc.forsale,"[cad, setup, sale, gplb, local, bu, motherboar..."


In [18]:
filters=[lambda x: x.lower(),strip_tags,strip_punctuation,strip_numeric,remove_stopwords,strip_short]
dataset_df['Clean_news1'] = dataset_df['News'].apply(lambda x: preprocess_string(x,filters))

In [19]:
dataset_df

,News,Label,Label_name,Clean_news,Clean_news1
0,gajarsky@pilot.njin.net writes:\n\nmorgan and ...,9,rec.sport.baseball,"[gajarski, pilot, njin, net, write, morgan, gu...","[gajarsky, pilot, njin, net, writes, morgan, g..."
1,"Well, I just got my Centris 610 yesterday. It...",4,comp.sys.mac.hardware,"[got, centri, yesterdai, took, week, place, or...","[got, centris, yesterday, took, weeks, placing..."
2,Archive-name: cryptography-faq/part10\nLast-mo...,11,sci.crypt,"[archiv, cryptographi, faq, modifi, faq, sci, ...","[archive, cryptography, faq, modified, faq, sc..."
3,> ATTENTION: Mac Quadra owners: Many storage i...,4,comp.sys.mac.hardware,"[attent, mac, quadra, owner, storag, industri,...","[attention, mac, quadra, owners, storage, indu..."
4,bobbe@vice.ICO.TEK.COM (Robert Beauchaine) wri...,0,alt.atheism,"[bobb, vice, ico, tek, com, robert, beauchain,...","[bobbe, vice, ico, tek, com, robert, beauchain..."
...,...,...,...,...,...
18841,\nWhy are circuit boards green? The material ...,12,sci.electronics,"[circuit, board, green, materi, goe, name, cir...","[circuit, boards, green, material, goes, names..."
18842,In article <1r941o$3tu@menudo.uh.edu> inde7wv@...,8,rec.motorcycles,"[articl, indewv, rosi, edu, write, bike, lucki...","[article, indewv, rosie, edu, writes, bike, lu..."
18843,We were told that the resolution on the 5FGe c...,4,comp.sys.mac.hardware,"[told, resolut, fge, anybodi, tri, run, higher...","[told, resolution, fge, anybody, tried, runnin..."
18844,CAD Setup For Sale:\n\nG486PLB Local Bus Mothe...,6,misc.forsale,"[cad, setup, sale, gplb, local, bu, motherboar...","[cad, setup, sale, gplb, local, bus, motherboa..."


In [20]:
dataset_dictionary = Dictionary(dataset_df['Clean_news1'])

In [21]:
len(dataset_dictionary)

96459

In [22]:
print(dataset_dictionary.token2id)

{'castillo': 0, 'cubs': 1, 'era': 2, 'gajarsky': 3, 'good': 4, 'guzman': 5, 'harkey': 6, 'hibbard': 7, 'higher': 8, 'idiots': 9, 'morgan': 10, 'net': 11, 'njin': 12, 'pilot': 13, 'pitch': 14, 'pitcher': 15, 'run': 16, 'stud': 17, 'think': 18, 'won': 19, 'writes': 20, 'year': 21, 'alright': 22, 'annoying': 23, 'apple': 24, 'appologized': 25, 'awful': 26, 'backlogged': 27, 'big': 28, 'boot': 29, 'buttons': 30, 'cable': 31, 'centri': 32, 'centris': 33, 'come': 34, 'complain': 35, 'connected': 36, 'control': 37, 'convenient': 38, 'deal': 39, 'dealer': 40, 'decided': 41, 'dissappionted': 42, 'drive': 43, 'ethernet': 44, 'exceptions': 45, 'exchange': 46, 'fast': 47, 'feature': 48, 'functions': 49, 'got': 50, 'great': 51, 'happy': 52, 'hard': 53, 'having': 54, 'heard': 55, 'high': 56, 'interupt': 57, 'keyboard': 58, 'know': 59, 'lack': 60, 'literature': 61, 'location': 62, 'loose': 63, 'machine': 64, 'mentioned': 65, 'nosy': 66, 'open': 67, 'order': 68, 'ordered': 69, 'partially': 70, 'pissed

In [23]:
dataset_corpus_bow = [dataset_dictionary.doc2bow(text) for text in dataset_df['Clean_news1']] #create a dataset corpus with bag of word vectorization

In [24]:
len(dataset_corpus_bow)

18846

In [25]:
print(dataset_corpus_bow[1])

[(22, 1), (23, 1), (24, 2), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 2), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 2), (44, 3), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 2), (52, 1), (53, 2), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 2), (66, 1), (67, 1), (68, 2), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 2), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 2), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1)]


In [26]:
tfidf = models.TfidfModel(dataset_corpus_bow)
dataset_corpus_tfidf = tfidf[dataset_corpus_bow]

In [27]:
len(dataset_corpus_tfidf)

18846

In [28]:
print(dataset_corpus_tfidf[1])

[(22, 0.12794312043780054), (23, 0.1032933602529823), (24, 0.1437906445046912), (25, 0.19446130648981633), (26, 0.09972437101248886), (27, 0.19446130648981633), (28, 0.056593976938038), (29, 0.09712742378308543), (30, 0.11391287593244794), (31, 0.08578843198010519), (32, 0.19446130648981633), (33, 0.10142837384435295), (34, 0.09687548915310225), (35, 0.10125120507490903), (36, 0.09133742977605598), (37, 0.061307837508357034), (38, 0.10891119725810347), (39, 0.06606330836365701), (40, 0.08855334717656915), (41, 0.07391023272465086), (42, 0.19446130648981633), (43, 0.11381832135728932), (44, 0.3110653713924378), (45, 0.11566217615948503), (46, 0.0969847936661337), (47, 0.0694228263237942), (48, 0.09383842074796034), (49, 0.09300779209433964), (50, 0.0455655162646588), (51, 0.10391838009790301), (52, 0.07696272067182856), (53, 0.10763740852872361), (54, 0.05222277834588352), (55, 0.05433687654103351), (56, 0.05404373403325169), (57, 0.15338363279035241), (58, 0.08979736303341844), (59, 0.

# Topic Modelling with Latent Dirichlet Allocation(LDA)

In [29]:
from gensim.models.ldamodel import LdaModel

In [30]:
lda_bow = LdaModel(dataset_corpus_bow,num_topics=20,id2word=dataset_dictionary,random_state=0)

In [31]:
lda_topics_bow = lda_bow.print_topics(num_words=8)
for topic in lda_topics_bow:
  print(topic)

(0, '0.013*"writes" + 0.011*"edu" + 0.009*"drive" + 0.009*"article" + 0.007*"like" + 0.007*"gamma" + 0.006*"battery" + 0.006*"time"')
(1, '0.031*"god" + 0.011*"jesus" + 0.009*"christ" + 0.008*"church" + 0.007*"sin" + 0.007*"bible" + 0.006*"paul" + 0.006*"christian"')
(2, '0.013*"government" + 0.012*"key" + 0.008*"chip" + 0.008*"clipper" + 0.008*"encryption" + 0.007*"law" + 0.007*"use" + 0.006*"keys"')
(3, '0.012*"space" + 0.008*"nasa" + 0.007*"writes" + 0.006*"article" + 0.006*"shuttle" + 0.006*"earth" + 0.005*"hst" + 0.005*"edu"')
(4, '0.012*"file" + 0.012*"edu" + 0.011*"image" + 0.010*"graphics" + 0.008*"ftp" + 0.008*"files" + 0.008*"available" + 0.007*"program"')
(5, '0.018*"cramer" + 0.017*"men" + 0.015*"gay" + 0.014*"homosexual" + 0.010*"sex" + 0.010*"clayton" + 0.009*"sexual" + 0.009*"kinsey"')
(6, '0.012*"conference" + 0.009*"echo" + 0.007*"xdm" + 0.007*"page" + 0.006*"incoming" + 0.005*"paris" + 0.004*"title" + 0.004*"perlman"')
(7, '0.010*"like" + 0.009*"know" + 0.008*"good" +

In [32]:
lda_tfidf = LdaModel(dataset_corpus_tfidf, id2word=dataset_dictionary, num_topics=20)

In [33]:
lda_topics_tfidf = lda_tfidf.print_topics(num_words=8)
for topic in lda_topics_tfidf:
  print(topic)

(0, '0.002*"photography" + 0.002*"capture" + 0.001*"iisi" + 0.001*"hitter" + 0.001*"harry" + 0.001*"pds" + 0.001*"allergy" + 0.001*"laserwriter"')
(1, '0.004*"cobb" + 0.002*"alexia" + 0.002*"leaf" + 0.001*"russ" + 0.001*"blah" + 0.001*"duo" + 0.001*"liter" + 0.001*"bake"')
(2, '0.002*"summarize" + 0.001*"bony" + 0.001*"jake" + 0.001*"dreams" + 0.001*"irq" + 0.001*"tammy" + 0.001*"joystick" + 0.001*"noems"')
(3, '0.003*"chronic" + 0.002*"infante" + 0.002*"seagate" + 0.002*"feustel" + 0.002*"janet" + 0.002*"atari" + 0.001*"timer" + 0.001*"bcci"')
(4, '0.002*"drivers" + 0.002*"emm" + 0.002*"dos" + 0.002*"diamond" + 0.002*"livesey" + 0.002*"windows" + 0.002*"turbo" + 0.002*"card"')
(5, '0.002*"pin" + 0.002*"pins" + 0.002*"external" + 0.002*"pit" + 0.002*"monitor" + 0.002*"det" + 0.002*"chi" + 0.002*"port"')
(6, '0.002*"mustang" + 0.002*"jpeg" + 0.002*"tiff" + 0.002*"kidney" + 0.002*"nixon" + 0.002*"hallam" + 0.001*"rider" + 0.001*"docs"')
(7, '0.002*"clutch" + 0.002*"batf" + 0.002*"childre

# Topic Modelling with Latent Semantic Analysis/Indexing(LSA/LSI)

In [34]:
from gensim.models.lsimodel import LsiModel

In [35]:
lsi_bow = LsiModel(corpus=dataset_corpus_bow,id2word=dataset_dictionary,num_topics=20)

In [36]:
lsi_topics_bow = lsi_bow.print_topics(num_words=8)
for topic in lsi_topics_bow:
  print(topic)

(0, '-0.994*"max" + -0.069*"giz" + -0.068*"bhj" + -0.025*"qax" + -0.015*"biz" + -0.014*"nrhj" + -0.014*"bxn" + -0.012*"nuy"')
(1, '0.255*"jpeg" + 0.253*"file" + 0.219*"edu" + 0.204*"image" + 0.204*"dos" + 0.164*"use" + 0.153*"available" + 0.137*"ftp"')
(2, '-0.772*"dos" + -0.277*"windows" + -0.147*"microsoft" + 0.113*"people" + -0.109*"tcp" + 0.090*"jpeg" + -0.090*"mouse" + 0.083*"know"')
(3, '0.435*"jpeg" + 0.231*"image" + -0.216*"people" + -0.174*"said" + -0.171*"dos" + 0.167*"file" + 0.166*"gif" + -0.166*"know"')
(4, '-0.442*"jpeg" + 0.297*"edu" + -0.190*"dos" + 0.171*"pub" + -0.158*"gif" + -0.140*"people" + 0.138*"com" + 0.138*"ftp"')
(5, '0.439*"god" + 0.366*"jehovah" + 0.297*"lord" + 0.275*"elohim" + -0.200*"file" + 0.183*"christ" + 0.177*"jesus" + 0.141*"father"')
(6, '-0.640*"file" + -0.204*"gun" + 0.164*"edu" + 0.162*"image" + -0.134*"jehovah" + -0.132*"god" + -0.115*"output" + -0.110*"control"')
(7, '0.350*"stephanopoulos" + 0.267*"president" + -0.221*"use" + 0.221*"file" + 0

In [37]:
lsi_tfidf = LsiModel(dataset_corpus_tfidf, id2word=dataset_dictionary, num_topics=20)

In [38]:
lsi_topics_tfidf = lsi_tfidf.print_topics(num_words=8)
for topic in lsi_topics_tfidf:
  print(topic)

(0, '0.127*"people" + 0.122*"god" + 0.110*"know" + 0.109*"like" + 0.101*"think" + 0.099*"windows" + 0.094*"edu" + 0.093*"use"')
(1, '-0.278*"god" + 0.244*"windows" + 0.193*"drive" + 0.170*"card" + 0.164*"scsi" + 0.152*"dos" + 0.125*"thanks" + -0.122*"jesus"')
(2, '-0.482*"god" + -0.198*"jesus" + 0.194*"game" + 0.136*"team" + -0.129*"bible" + -0.126*"windows" + -0.122*"scsi" + 0.120*"games"')
(3, '-0.258*"game" + 0.241*"key" + -0.216*"drive" + -0.191*"scsi" + 0.172*"encryption" + 0.169*"clipper" + 0.168*"chip" + -0.164*"team"')
(4, '0.381*"scsi" + 0.364*"drive" + -0.316*"windows" + 0.199*"ide" + -0.146*"file" + 0.139*"controller" + 0.128*"drives" + 0.124*"chip"')
(5, '0.276*"key" + 0.274*"god" + 0.214*"chip" + -0.212*"israel" + 0.207*"game" + 0.170*"clipper" + 0.167*"encryption" + 0.139*"keys"')
(6, '0.309*"car" + -0.253*"windows" + -0.210*"israel" + -0.205*"scsi" + -0.192*"dos" + -0.135*"game" + 0.132*"bike" + -0.125*"israeli"')
(7, '0.435*"card" + -0.265*"drive" + 0.221*"video" + 0.20

# Topic Modelling Visualization with pyLDAvis

In [39]:
%%capture
!pip install pyLDAvis

In [46]:
import pyLDAvis
import pyLDAvis.gensim_models

In [47]:
pyLDAvis.enable_notebook()

In [50]:
vis_bow = pyLDAvis.gensim_models.prepare(lda_bow, dataset_corpus_bow, dataset_dictionary)
vis_bow

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14    -0.180205  0.067994       1        1  12.686874
4      0.167075  0.190427       2        1   9.887234
7     -0.004118  0.066002       3        1   8.925347
11    -0.172302 -0.012892       4        1   7.708848
17     0.136189  0.141629       5        1   6.866600
19    -0.065509  0.021868       6        1   6.215635
16    -0.009175 -0.057871       7        1   5.209632
1     -0.153669  0.036146       8        1   5.074315
9     -0.135337 -0.029723       9        1   5.048693
2     -0.032442  0.088467      10        1   5.029165
13    -0.024265  0.043209      11        1   4.738789
15    -0.000989 -0.174073      12        1   4.161186
8      0.085510  0.118396      13        1   3.887261
3      0.014071 -0.097657      14        1   3.802398
0      0.044942  0.043218      15        1   3.094085
10    -0.105217 -0.034757      16        1   2.771050
12     0.207324 -0.183031      17        1   1.772455
18     0.099743  0.014600      18        1   1.450803
5      0.010118 -0.149188      19        1   0.837395
6      0.118255 -0.092765      20        1   0.832235, topic_info=         Term          Freq         Total Category  logprob  loglift
2109      god   5200.000000   5200.000000  Default  30.0000  30.0000
1249     game   2682.000000   2682.000000  Default  29.0000  29.0000
20     writes  14727.000000  14727.000000  Default  28.0000  28.0000
986   windows   3373.000000   3373.000000  Default  27.0000  27.0000
308      file   2850.000000   2850.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
743      york     43.632244    563.895183  Topic20  -5.9812   2.2297
886       use     52.230798   6522.032843  Topic20  -5.8014  -0.0385
279       edu     53.359184  12350.390391  Topic20  -5.7800  -0.6556
509    number     46.326868   2779.747216  Topic20  -5.9213   0.6944
20     writes     43.841559  14727.649347  Topic20  -5.9764  -1.0281

[1583 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
6038       1  0.798487  absolute
6038       2  0.023717  absolute
6038       3  0.023717  absolute
6038       4  0.019765  absolute
6038       6  0.003953  absolute
...      ...       ...       ...
743       19  0.005320      york
743       20  0.078029      york
25530     16  0.960129      yuri
15332     14  0.983697   yzerman
19409     19  0.926560    zipper

[8581 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 5, 8, 12, 18, 20, 17, 2, 10, 3, 14, 16, 9, 4, 1, 11, 13, 19, 6, 7])

In [51]:
vis_tfidf = pyLDAvis.gensim_models.prepare(lda_tfidf, dataset_corpus_tfidf, dataset_dictionary)
vis_tfidf

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10    -0.381378 -0.024369       1        1  47.603728
18    -0.144915 -0.101181       2        1  10.919142
7     -0.115756  0.167174       3        1   9.370744
11    -0.055210  0.062300       4        1   6.894738
14     0.005791 -0.055611       5        1   3.348592
4      0.001990 -0.021482       6        1   2.545908
6      0.017685  0.002321       7        1   2.540410
5      0.025555 -0.020735       8        1   1.815405
0      0.037389  0.000489       9        1   1.782659
8      0.036359 -0.005606      10        1   1.620811
17     0.048670 -0.000775      11        1   1.404766
19     0.053624 -0.003152      12        1   1.285679
3      0.053761 -0.001326      13        1   1.251387
2      0.051807 -0.003528      14        1   1.172182
16     0.061599  0.000563      15        1   1.097198
13     0.057207  0.000924      16        1   1.078069
1      0.061771  0.001378      17        1   1.076644
15     0.060641 -0.000498      18        1   1.075376
9      0.060646  0.001623      19        1   1.058498
12     0.062764  0.001491      20        1   1.058063, topic_info=          Term        Freq       Total Category  logprob  loglift
986    windows  121.000000  121.000000  Default  30.0000  30.0000
5253      card   70.000000   70.000000  Default  29.0000  29.0000
1978       dos   72.000000   72.000000  Default  28.0000  28.0000
308       file   69.000000   69.000000  Default  27.0000  27.0000
1249      game  107.000000  107.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
9209       cbc    0.545758    1.806631  Topic20  -7.7525   3.3517
1311     prism    0.835068    7.114539  Topic20  -7.3271   2.4063
1306    gatech    0.698669    9.713986  Topic20  -7.5055   1.9166
11750  finland    0.592034    5.228622  Topic20  -7.6711   2.3704
2642    france    0.568145    5.004296  Topic20  -7.7123   2.3731

[966 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
29887     11  0.747539  abomination
4449       3  0.963394        abuse
11778      7  0.755173         acid
24278     13  1.027058        acpub
12161      6  0.568219      adapter
...      ...       ...          ...
21         4  0.154176         year
9897      10  0.988025          yfn
9898      10  0.957356          ysu
23064     10  0.521786         yuan
18951      4  0.899052      zionism

[1450 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 19, 8, 12, 15, 5, 7, 6, 1, 9, 18, 20, 4, 3, 17, 14, 2, 16, 10, 13])